In [ ]:
import requests
import csv
import os
import pandas as pd

# Function that extracts features for neighbors based on the neighbor number, passed
# latitude and longitude, name (city refernece point). Also passes rows, which is the
# collection of data to export to CSV after collecting all data
def extract_neighbor_data(latitude, longitude, neighbor_number, name, rows):
  params = {
    'start': '20000101',
    'end': '20221231',
    'latitude': latitude,
    'longitude': longitude,
    'community': 'AG',
    'parameters': 'WS2M,T2MDEW,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN',
    'format': 'JSON'
  }

  # API call
  response = requests.get(base_url, params=params)
  data = response.json()

  # Set features we select (not exported from NASA DAV)
  header = ['Date', 'Neighbor Number', 'Reference', 'Latitude', 'Longitude']

  # Extract the feature names (keys in 'parameter') for the columns
  for feature_name in data['properties']['parameter'].keys():
    header.append(feature_name)
  header.append("FROST")

  # Find all the unique dates by iterating over the parameter data
  dates = set()
  for feature_name, date_val_dict in data['properties']['parameter'].items():
    for date in date_val_dict.keys():
      dates.add(date)

  # Sort dates to maintain chronological order
  dates = sorted(dates)

  # For each date, create a row with values for each feature
  for date in dates:
    # Fill in date, neighbor number, reference name, latitude, and longitude values
    row = [date, neighbor_number, name, latitude, longitude]
    # Add features to data
    for feature_name in data['properties']['parameter'].keys():
        # Get the value for the feature for the current date
        value = data['properties']['parameter'][feature_name].get(date, None)
        row.append(value)

        # Now we want to add the frost prediction output
        if feature_name == 'T2M_MIN':
            t2m = value
        elif feature_name == 'T2MDEW':
            dew = value

    # Check if dew < 0, and if dew < t2m
    if t2m < 0 and dew > t2m and dew < 0:
        frost = 1
    else:
        frost = 0

    # Add frost prediction to the row
    row.append(frost)
    # Add instance to the API query dataset
    rows.append(row)

  # Return all new data that was added on top of old data
  return rows

# Change directory to where you want files saved
os.chdir("MY_DIRECTORY")

# API URL
base_url = 'https://power.larc.nasa.gov/api/temporal/daily/point'

# Locations to query for: latitude, longitude, and city reference name
locations = [(35.7796, -78.6382, "Raleigh"), \
             (38.9072, -77.0369, "DC"), \
             (42.355, -71.0565, "Boston"), \
             (34.0008, -81.0351, "Columbia"), \
             (34.7501, -84.3885, "Atlanta"), \
             (40.7182, -74.006, "NYC"), \
             (39.7691, -86.158, "Indianapolis"), \
             (29.7601, -95.3701, "Houston")]

# Go through each location set to query
for loc in locations:
  # Assign latitude, longitude, and name of city
  latitude = loc[0]
  longitude = loc[1]
  name = loc[2]

  # Define parameters for query
  params = {
    'start': '20000101',
    'end': '20221231',
    'latitude': latitude,
    'longitude': longitude,
    'community': 'AG',
    'parameters': 'WS2M,T2MDEW,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN',
    'format': 'JSON'
  }

  # API call
  response = requests.get(base_url, params=params)
  data = response.json()

  # Set features we select (not exported from NASA DAV)
  header = ['Date', 'Neighbor Number', 'Reference', 'Latitude', 'Longitude']
  # Extract the feature names (keys in 'parameter') for the columns
  for feature_name in data['properties']['parameter'].keys():
    header.append(feature_name)
  header.append("FROST")

  # Prepare a list to hold the rows of the CSV
  rows = []

  # Find all the unique dates by iterating over the parameter data
  dates = set()
  for feature_name, date_val_dict in data['properties']['parameter'].items():
    for date in date_val_dict.keys():
      dates.add(date)

  # Sort dates to maintain chronological order
  dates = sorted(dates)

  # For each date, create a row with values for each feature
  for date in dates:
    # Fill in date, neighbor number (this is for the middle of the grid, so 5),reference name, latitude, and longitude values
    row = [date, 5, name, latitude, longitude]
    # Add features to data
    for feature_name in data['properties']['parameter'].keys():
        # Get the value for the feature for the current date
        value = data['properties']['parameter'][feature_name].get(date, None)
        row.append(value)

        # Now we want to add the frost prediction output
        if feature_name == 'T2M_MIN':
            t2m = value
        elif feature_name == 'T2MDEW':
            dew = value

    # Check if dew < 0, and if dew < t2m
    if t2m < 0 and dew > t2m and dew < 0:
        frost = 1
    else:
        frost = 0

    # Add frost prediction to instance
    row.append(frost)
    # Add instance to the dataset
    rows.append(row)

  # Now we do this for each neighbor on the tile. We'll use a 3x3 neighborhood
  rows = extract_neighbor_data(latitude - 0.5, longitude + 0.625, 1, name, rows)
  rows = extract_neighbor_data(latitude, longitude + 0.625, 2, name, rows)
  rows = extract_neighbor_data(latitude + 0.5, longitude + 0.625, 3, name, rows)
  rows = extract_neighbor_data(latitude - 0.5, longitude, 4, name, rows)
  rows = extract_neighbor_data(latitude + 0.5, longitude, 6, name, rows)
  rows = extract_neighbor_data(latitude - 0.5, longitude - 0.625, 7, name, rows)
  rows = extract_neighbor_data(latitude, longitude - 0.625, 8, name, rows)
  rows = extract_neighbor_data(latitude + 0.5, longitude - 0.625, 9, name, rows)

  # File name - Each city has an output file. These will be combined in another notebook
  filename = f"{name}_output.csv"

  # Open a CSV file for writing
  with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header (features as columns and date as the first column)
    writer.writerow(header)

    # Write the rows (values for each date)
    writer.writerows(rows)